In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- Selected Channels: 15 (Fp1, Fp2, F3, F4, Cz, P3, P4, C3, C4, Cz, P3, P4, Pz, O1, O2)
- Range of Delta (for Artificial Data Generation): [0.0001, 0.0005]
- Input size: (65, 198, 15)
- Input type: Short Time Fourier Transform with window length 128, overlap 100
- Number of samples: 1212 (1200 fake samples + 12 real samples)

In [2]:
import numpy as np

path = '/content/drive/My Drive/ML/'
stft = np.load(path + 'Fstft3.npy')

In [3]:
num_classes = 4

output_values = np.concatenate((np.tile(0, 303), np.tile(1, 303), np.tile(2, 303), np.tile(3, 303)))
output_values = np.eye(num_classes)[output_values]

data_array = stft

In [4]:
data_array.shape

(1212, 15, 65, 198)

In [5]:
import keras

In [6]:
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

In [7]:
# Parameters
train_params = {'dim': (65,198),
                'batch_size': 64,
                'n_classes':4,
                'n_channels': 15,
                'shuffle': True,
                'input_type': 'stft',
                'augment': True
               }
val_params = {'dim': (65,198),
              'batch_size': 64,
              'n_classes':4,
              'n_channels': 15,
              'shuffle': False,
              'input_type': 'stft',
              'augment': False
             }

In [11]:
import tensorflow as tf

def mish(inputs):
  x = tf.nn.softplus(inputs)
  x = tf.nn.tanh(x)
  x = tf.multiply(x, inputs)
  return x


stft_input = keras.Input(shape=(65, 198, 15))
x = Conv2D(32, 3, activation=mish)(stft_input)
x = BatchNormalization()(x)
x = Conv2D(32, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(32, 5, strides=2, padding='same', activation=mish)(x)
x = Dropout(0.4)(x)

x = Conv2D(64, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 5, strides=2, padding='same', activation=mish)(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Conv2D(128, 4, activation=mish)(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dropout(0.4)(x)

stft_prediction = Dense(4, activation='softmax')(x)

stft_model = Model(stft_input, stft_prediction)

stft_model.compile(optimizer = 'adam',
                    loss='categorical_crossentropy',
                    metrics=["accuracy"])

stft_model.summary()

stft_annealer = LearningRateScheduler(lambda x: 1e-3 * 0.97 ** x)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 65, 198, 15)]     0         
                                                                 
 conv2d_7 (Conv2D)           (None, 63, 196, 32)       4352      
                                                                 
 batch_normalization_6 (Batc  (None, 63, 196, 32)      128       
 hNormalization)                                                 
                                                                 
 conv2d_8 (Conv2D)           (None, 61, 194, 32)       9248      
                                                                 
 batch_normalization_7 (Batc  (None, 61, 194, 32)      128       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 31, 97, 32)        2563

In [12]:
from sklearn.model_selection import train_test_split

data_array.shape

input = np.transpose(data_array[:, :, :, :], (0, 2, 3, 1))
print(input.shape)

X_train_temp, X_test, Y_train_temp, Y_test = train_test_split(input, output_values, test_size=0.3, random_state=42)
X_train, X_validate, Y_train, Y_validate = train_test_split(X_train_temp, Y_train_temp, test_size=0.25, random_state=36)

(1212, 65, 198, 15)


In [13]:
stft_history = stft_model.fit(X_train, Y_train, batch_size = 64, epochs = 100,
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[stft_annealer], class_weight = {})

Epoch 1/100
10/10 - 129s - loss: 2.2289 - accuracy: 0.3050 - val_loss: 1.5039 - val_accuracy: 0.2689 - lr: 0.0010 - 129s/epoch - 13s/step
Epoch 2/100
10/10 - 122s - loss: 2.0908 - accuracy: 0.4277 - val_loss: 2.2010 - val_accuracy: 0.2689 - lr: 9.7000e-04 - 122s/epoch - 12s/step
Epoch 3/100
10/10 - 118s - loss: 1.4937 - accuracy: 0.6258 - val_loss: 3.1067 - val_accuracy: 0.2689 - lr: 9.4090e-04 - 118s/epoch - 12s/step
Epoch 4/100
10/10 - 110s - loss: 0.6441 - accuracy: 0.8381 - val_loss: 3.1274 - val_accuracy: 0.2689 - lr: 9.1267e-04 - 110s/epoch - 11s/step
Epoch 5/100
10/10 - 122s - loss: 0.1131 - accuracy: 0.9575 - val_loss: 3.4782 - val_accuracy: 0.2689 - lr: 8.8529e-04 - 122s/epoch - 12s/step
Epoch 6/100
10/10 - 120s - loss: 0.0027 - accuracy: 0.9984 - val_loss: 3.6630 - val_accuracy: 0.2689 - lr: 8.5873e-04 - 120s/epoch - 12s/step
Epoch 7/100
10/10 - 120s - loss: 0.0021 - accuracy: 1.0000 - val_loss: 3.9557 - val_accuracy: 0.2689 - lr: 8.3297e-04 - 120s/epoch - 12s/step
Epoch 8/10

In [ ]:
test_loss, test_accuracy = stft_model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 1.0
